In [1]:
import numpy as np, pandas as pd, os, sys
sys.path.append(r'D:\Tools\DIgSILENT\PowerFactory 2023 SP5\Python\3.7')
import powerfactory as pf
import powfacpy as pfp
import platform

## Get_obj in large grids
according to issues #38 [get_obj() on very large grids](https://github.com/FraunhIEE-UniKassel-PowSysStability/powfacpy/issues/38) and #49 [Performance on scripts](https://github.com/FraunhIEE-UniKassel-PowSysStability/powfacpy/issues/49), there are performance differences between multiple object retrieval functionalities when operating on grids with a large node number.
The runtime of the following functions are listed in increasing order; according to the issues.
| Functionality                  | Description            |
| :------------------------------| ---------------------- |
| _Network Manager_              | Integrated into PF GUI |
| `app.GetCalcRelevantObjects()` | Only retrieves objects that are relevent for calculations (in active study cases, grids and variations)                       |
| PowFacPy's `get_obj()`         | Gets objects based on paths, wildcards and conditions.                       |

This notebook contains an investigation of the matter. As the large test grid, the same model is used as was in the issues.

In [2]:
# Options and parameters
proj_name = "20231114_0822_BtB_QStep"

# System parameters
print("="*10, "System Information", "="*10)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

========== System Information ==========
System: Windows
Release: 10
Version: 10.0.19041
Machine: AMD64
Processor: AMD64 Family 25 Model 68 Stepping 1, AuthenticAMD


In [3]:
app = pf.GetApplication()
pfbi = pfp.PFBaseInterface(app)
pfbi.app.ActivateProject(proj_name)
app.Show()

In [4]:
terminals_calcrelevant = pfbi.app.GetCalcRelevantObjects('*.ElmTerm')
print(f"Amount of Nodes/Terminals when using GetCalcRelevantObjects: {len(terminals_calcrelevant)}")
terminals_pfp = pfbi.get_obj("*.ElmTerm")
print(f"Amount of Nodes/Terminals when using get_ojb: {len(terminals_pfp)}")

Amount of Nodes/Terminals when using GetCalcRelevantObjects: 12157
Amount of Nodes/Terminals when using get_ojb: 12157


In [6]:
all_objs_calcrelevant = pfbi.app.GetCalcRelevantObjects()
print(f"Amount of objects when using GetCalcRelevantObjects: {len(all_objs_calcrelevant)}")
all_objs_pfp = pfbi.get_obj("*")
print(f"Amount of objects when using get_obj: {len(all_objs_pfp)}")

Amount of objects when using GetCalcRelevantObjects: 128858
Amount of objects when using get_obj: 484515


#### Without App shown

In [7]:
print("="*10 + " Terminals " + "="*10)
print("get_obj:  ", end = "")
%timeit pfbi.get_obj("*.ElmTerm")
print("calc_rel: ", end="")
%timeit pfbi.app.GetCalcRelevantObjects('*.ElmTerm')

print("="*9 + " All Objects " + "="*9)
print("get_obj:  ", end = "")
%timeit pfbi.get_obj("*")
print("calc_rel: ", end="")
%timeit pfbi.app.GetCalcRelevantObjects()

========== Terminals ==========
get_obj:  287 ms ± 7.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
calc_rel: 186 ms ± 4.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
========= All Objects =========
get_obj:  387 ms ± 8.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
calc_rel: 191 ms ± 3.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def count_subnets():
   terminals = app.GetCalcRelevantObjects('*.ElmTerm')
   counter = 0
   for terminal in terminals:
      if not terminal.HasAttribute('b:ipat'):
         counter += 1
%timeit count_subnets()

603 ms ± 29.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
def get_obj_composition_1(exclude_all=False):
    all_objs_cls = [i.GetClassName() for i in pfbi.get_obj('*')]

    composition = {}
    for unique_cls in np.unique(all_objs_cls):
        composition[unique_cls[:3]] = len(pfbi.get_obj(f"*.{unique_cls}*"))

    return composition

def get_obj_composition_2(exclude_all=False):
    composition = {
        'int': len(pfbi.get_obj('*.Int*', error_if_non_existent=False)),
        'elm': len(pfbi.get_obj('*.Elm*', error_if_non_existent=False)),
        'typ': len(pfbi.get_obj('*.Typ*', error_if_non_existent=False)),
        'sta': len(pfbi.get_obj('*.Sta*', error_if_non_existent=False)),
        'set': len(pfbi.get_obj('*.Set*', error_if_non_existent=False)),
        'com': len(pfbi.get_obj('*.Com*', error_if_non_existent=False)),
        'cha': len(pfbi.get_obj('*.Cha*', error_if_non_existent=False)),
        'blk': len(pfbi.get_obj('*.Blk*', error_if_non_existent=False)),
        'cim': len(pfbi.get_obj('*.Cim*', error_if_non_existent=False)),
        'evt': len(pfbi.get_obj('*.Evt*', error_if_non_existent=False)),
        'opt': len(pfbi.get_obj('*.Opt*', error_if_non_existent=False)),
        'plt': len(pfbi.get_obj('*.Plt*', error_if_non_existent=False)),
        'grp': len(pfbi.get_obj('*.Grp*', error_if_non_existent=False)),
        'scn': len(pfbi.get_obj('*.Scn*', error_if_non_existent=False)),
        'vis': len(pfbi.get_obj('*.Vis*', error_if_non_existent=False)),
    }

    if not exclude_all:
        composition['all'] = len(pfbi.get_obj('*'))

    return composition

%timeit get_obj_composition_2()
%timeit get_obj_composition_1()

4.8 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 3s ± 1.74 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
all_objs = pfbi.get_obj('*')
all_cls = [i.GetClassName() for i in all_objs]
print("="*9 + " All Objects " + "="*9)
print("get_obj:                ", end = "")
%timeit pfbi.get_obj("*")
print("getclassname list comp: ", end="")
%timeit [i.GetClassName() for i in all_objs]
print("numpy unique:           ", end="")
%timeit np.unique(all_cls)

========= All Objects =========
get_obj:                366 ms ± 9.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
getclassname list comp: 19.7 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy unique:           117 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### With App shown

In [4]:
print("="*10 + " Terminals " + "="*10)
print("get_obj:  ", end = "")
%timeit pfbi.get_obj("*.ElmTerm")
print("calc_rel: ", end="")
%timeit pfbi.app.GetCalcRelevantObjects('*.ElmTerm')

print("="*9 + " All Objects " + "="*9)
print("get_obj:  ", end = "")
%timeit pfbi.get_obj("*")
print("calc_rel: ", end="")
%timeit pfbi.app.GetCalcRelevantObjects()

========== Terminals ==========
get_obj:  303 ms ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
calc_rel: 198 ms ± 6.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
========= All Objects =========
get_obj:  376 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
calc_rel: 186 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Conclusion
The difference between `GetCalcRelevantObjects` and `get_obj()` seems to be insignificant. Receiving elements with each method in a PowerFactory project with around $500.000$ objects takes only several milliseconds. Showing or hiding the PF GUI had no influence. The notebook is not able to reproduce the time difference described in the github issues.

The notebook observes a more significant time difference when using methods of PowerFactory's `DataObject` class. The function `get_obj_composition_1` sorts all objects based on their top-level type, using the `GetClassName` method. Function `get_obj_composition_2` implements the same functionality, however it only uses the `get_obj` function. On average, `get_obj_composition_1` is one minute slower than its counterpart. A major reason for this slow-down is the `GetClassName` method, which takes around 20 seconds.

In conclusion, reasons for the time difference in both `get_obj` and `GetCalcRelevantObjects` could be:
* Inherent search filter for `GetCalcRelevantObjects`.
* Performance differences in executing machine and network speed (the original problems occured on a VM).
* Implementation of the PowerFactory Python Interface, specifically for the `DataObject` class (Receiving objects seems to be implemented well).
* Speed of list comprehensions or for-loops.